# Upload sortings for biophy npx spont

author: laquitainesteeve@gmail.com

Execution time: < 1 min

Special hardware: on CPU, does not require GPU.

# Setup 

Activate demo virtual environment (envs/demo.yml)

```bash
python -m ipykernel install --user --name demo --display-name "demo"
```

In [38]:
%%time 

# import python packages
import os
import numpy as np
from time import time
from dandi.dandiapi import DandiAPIClient
import spikeinterface.extractors as se
import spikeinterface.sorters as ss
import spikeinterface
import spikeinterface as si
from pynwb.file import NWBFile, Subject
from pynwb import NWBHDF5IO
import uuid
from datetime import datetime
from dateutil.tz import tzlocal
from neuroconv.tools.spikeinterface import add_recording_to_nwbfile, add_sorting_to_nwbfile
print("spikeinterface", spikeinterface.__version__)

# setup project
proj_path = "/home/steeve/steeve/epfl/code/spikebias/"
os.chdir(proj_path)

# setup pipeline parameters
dandiset_id = '001250'

# setup nwb files save paths
nwb_file_path = os.path.join(proj_path, "tmp/")

# setup sorters names
sorter_name = ["ks4", "ks3", "ks25", "ks2", "ks", "hs", "ks4_10m", "ks3_10m", "ks25_10m", "ks2_10m", "ks_10m", "hs_10m"] # full and 10 min sortings

spikeinterface 0.101.2
CPU times: user 997 μs, sys: 13 μs, total: 1.01 ms
Wall time: 1.03 ms


## Custom functions

In [ ]:
def load_sortings(sorting_folders, sorting_path):
    
    # load sorting extractors
    folder_paths = []
    sorting_list = []
    for folder_i in sorting_folders:
        folder_paths = os.path.join(sorting_path, folder_i)
        sorting_list.append(si.load_extractor(folder_paths))
    
    # unit-test
    assert len(sorting_list) == len(sorting_folders), "sorting files are missing"
    return sorting_list


def upload_sorting_nwb_to_dandi(sorting_list, dandiset_id, subject_id, nwb_file_path, session_description, experiment_description):
    """
    IMPORTANT! For Dandi 0.69.3, file_path must follow DANDI convention: {/sub-<subject_id>/sub-<subject_id>_ses-<session_id>.nwb}
    """
    # create nwb files save path
    subject_path = os.path.join(nwb_file_path, f"{dandiset_id}/sub-{subject_id}/")
    os.makedirs(subject_path, exist_ok=True)

    # loop over SortingExtractor files
    for s_ix, sorter_i in enumerate(sorter_name):
        
        # file path
        file_path = os.path.join(subject_path, f"sub-{subject_id}_ses-{sorter_i}.nwb")

        # add nwb session metadata (child folder)
        nwbfile = NWBFile(
            session_description=session_description,
            identifier=str(uuid.uuid4()),
            session_start_time=datetime.now(tzlocal()),
            experimenter="Laquitaine, Steeve",
            lab="Blue Brain Project",
            institution="EPFL",
            experiment_description=experiment_description,
            session_id=sorter_i,
            related_publications="https://doi.org/10.1101/2024.12.04.626805",
            keywords=["Biophysical simulation", "dense extracellular recordings", "spike sorting"]
        )

        # add subject metadata (parent folder)
        nwbfile.subject = Subject(
            subject_id = subject_id,
            species = "Rattus norvegicus",
            age = "P14D",
            sex = "M",
            description = "Wistar Rat",
        )

        # add SortingExtractor to nwb
        add_sorting_to_nwbfile(sorting = sorting_list[s_ix], nwbfile = nwbfile)

        # write nwb file locally
        with NWBHDF5IO(path=file_path, mode="w") as io:
            io.write(nwbfile)

        # WARNINg: check file with nwbinspector <source_folder> --config dandi beforehand

    # upload nwb to dandiset
    os.system(
        f"""
        export DANDI_API_KEY='210e68743286d64e84743bd8980d5771ef82bf4d';
        cd {dandiset_id};
        dandi organize {subject_path} -f dry;
        dandi organize {subject_path};    
        dandi upload
        """
    )

    # delete temporary datasets
    os.system(f"rm -rf {nwb_file_path}")
    os.system(f"rm -rf {dandiset_id}")

## Initialize dandiset

In [ ]:
%%time 

# download a local dandiset folder with yaml config
os.system("dandi download --download dandiset.yaml DANDI:001250")

PATH                 SIZE DONE    DONE% CHECKSUM STATUS    MESSAGE    
001250/dandiset.yaml                             skipped   no change  
Summary:                  0 Bytes                1 skipped 1 no change
                          <0.00%                                      


2025-06-15 19:04:45,932 [    INFO] Logs saved in /home/steeve/.local/state/dandi-cli/log/2025.06.15-17.04.45Z-1723958.log


0

## NPX_SPONT

In [44]:
%%time

# setup experiment parameters
subject_id = "001-fitted"
sorting_path = os.path.join(proj_path, "dataset/01_intermediate/sorting/0_silico/npx_spont/")
sorting_folders = ["SortingKS4", "SortingKS3", "SortingKS2_5", "SortingKS2", "SortingKS", "SortingHerdingspikes", "SortingKS4_10m", "SortingKS3_10m", "SortingKS2_5_10m", "SortingKS2_10m", "SortingKS_10m", "SortingHerdingspikes_10m"]

# load list of sorting extractors
sorting_list = load_sortings(sorting_folders, sorting_path)

# upload the sorting nwb files to dandi archive
upload_sorting_nwb_to_dandi(sorting_list, dandiset_id, subject_id, nwb_file_path, session_description = "Biophysical simulation of neuropixels in the spontaneous regime. Background noise and gain are fitted to Marques-Smith dataset.", experiment_description="Biophysical simulation of neuropixels in the spontaneous regime. Background noise and gain are fitted to Marques-Smith dataset.")

/home/steeve/steeve/epfl/code/spikebias/envs/demo/lib/python3.10/site-packages/spikeinterface/core/base.py:1129: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.100.6 is recommended
  warnings.warn(
/home/steeve/steeve/epfl/code/spikebias/envs/demo/lib/python3.10/site-packages/spikeinterface/core/base.py:1129: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.96.1 is recommended
  warnings.warn(
/home/steeve/steeve/epfl/code/spikebias/envs/demo/lib/python3.10/site-packages/spikeinterface/core/base.py:1129: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.100.5 is recommended
  warnings.warn(
2025-06-15 19:05:10,847 [    INFO] Loading metadata from 12 files
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 

DRY: /home/steeve/steeve/epfl/code/spikebias/tmp/001250/sub-001-fitted/sub-001-fitted_ses-ks_10m.nwb -> sub-001-fitted/sub-001-fitted_ses-ks-10m.nwb
DRY: /home/steeve/steeve/epfl/code/spikebias/tmp/001250/sub-001-fitted/sub-001-fitted_ses-ks4.nwb -> sub-001-fitted/sub-001-fitted_ses-ks4.nwb
DRY: /home/steeve/steeve/epfl/code/spikebias/tmp/001250/sub-001-fitted/sub-001-fitted_ses-ks2_10m.nwb -> sub-001-fitted/sub-001-fitted_ses-ks2-10m.nwb
DRY: /home/steeve/steeve/epfl/code/spikebias/tmp/001250/sub-001-fitted/sub-001-fitted_ses-ks2.nwb -> sub-001-fitted/sub-001-fitted_ses-ks2.nwb
DRY: /home/steeve/steeve/epfl/code/spikebias/tmp/001250/sub-001-fitted/sub-001-fitted_ses-ks.nwb -> sub-001-fitted/sub-001-fitted_ses-ks.nwb
DRY: /home/steeve/steeve/epfl/code/spikebias/tmp/001250/sub-001-fitted/sub-001-fitted_ses-ks3_10m.nwb -> sub-001-fitted/sub-001-fitted_ses-ks3-10m.nwb
DRY: /home/steeve/steeve/epfl/code/spikebias/tmp/001250/sub-001-fitted/sub-001-fitted_ses-ks3.nwb -> sub-001-fitted/sub-00

2025-06-15 19:05:11,718 [    INFO] Organized 0 out of 12 paths. Visit /home/steeve/steeve/epfl/code/spikebias/001250/
2025-06-15 19:05:11,718 [    INFO] Logs saved in /home/steeve/.local/state/dandi-cli/log/2025.06.15-17.05.10Z-1724577.log
2025-06-15 19:05:12,925 [    INFO] Loading metadata from 12 files
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done   3 out of  12 | elapsed:    0.6s remaining:    1.9s
[Parallel(n_jobs=-1)]: Done   5 out of  12 | elapsed:    0.6s remaining:    0.9s
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:    0.6s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done   9 out of  12 | elapsed:    0.7s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    0.8s finished
2025-06-15 19:05:13,732 [    INFO] Symlink support autodetected; setting files_mode='symlink'
2025-06-15 19:05:13,737 [    INFO] Organized 12 paths. Visit /home

PATH                                           SIZE     ERRORS PROGRESS STATUS                MESSAGE                        
dandiset.yaml                                  5.6 kB                   skipped               should be edited online        
sub-001-fitted/sub-001-fitted_ses-hs-10m.nwb   988.8 kB   0        100% done                  exists (older) - reuploading   
sub-001-fitted/sub-001-fitted_ses-hs.nwb       3.1 MB     0        100% done                  exists (older) - reuploading   
sub-001-fitted/sub-001-fitted_ses-ks-10m.nwb   4.7 MB     0        100% done                  exists (older) - reuploading   
sub-001-fitted/sub-001-fitted_ses-ks.nwb       14.7 MB    0        100% done                  exists (older) - reuploading   
sub-001-fitted/sub-001-fitted_ses-ks2-10m.nwb  3.0 MB     0        100% done                  exists (older) - reuploading   
sub-001-fitted/sub-001-fitted_ses-ks2.nwb      9.9 MB     0        100% done                  exists (older) - reuploa

2025-06-15 19:05:25,192 [    INFO] Logs saved in /home/steeve/.local/state/dandi-cli/log/2025.06.15-17.05.14Z-1724963.log


## NPX_EVOKED

In [45]:
%%time

# setup experiment parameters
subject_id = "002-fitted"
sorting_path = os.path.join(proj_path, "dataset/01_intermediate/sorting/0_silico/npx_evoked/")
sorting_folders = ["SortingKS4_full", "SortingKS3_full", "SortingKS2_5_full", "SortingKS2_full", "SortingKS_full", "SortingHerdingspikes_full", "SortingKS4_10m", "SortingKS3_10m", "SortingKS2_5_10m", "SortingKS2_10m", "SortingKS_10m", "SortingHerdingspikes_10m"]

# load list of sorting extractors
sorting_list = load_sortings(sorting_folders, sorting_path)

# upload the sorting nwb files to dandi archive
upload_sorting_nwb_to_dandi(sorting_list, dandiset_id, subject_id, nwb_file_path, session_description = "Biophysical simulation of neuropixels in the evoked regime. Background noise and gain are fitted to Marques-Smith dataset.", experiment_description="Biophysical simulation of neuropixels in the evoked regime. Background noise and gain are fitted to Marques-Smith dataset.")

/home/steeve/steeve/epfl/code/spikebias/envs/demo/lib/python3.10/site-packages/spikeinterface/core/base.py:1129: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.100.6 is recommended
  warnings.warn(
/home/steeve/steeve/epfl/code/spikebias/envs/demo/lib/python3.10/site-packages/spikeinterface/core/base.py:1129: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.100.5 is recommended
  warnings.warn(
2025-06-15 19:07:41,203 [    INFO] Loading metadata from 12 files
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done   3 out of  12 | elapsed:    0.7s remaining:    2.1s
[Parallel(n_jobs=-1)]: Done   5 out of  12 | elapsed:    0.7s remaining:    1.0s
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:    0.7s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done   9 out of  

DRY: /home/steeve/steeve/epfl/code/spikebias/tmp/001250/sub-002-fitted/sub-002-fitted_ses-ks2_10m.nwb -> sub-002-fitted/sub-002-fitted_ses-ks2-10m.nwb
DRY: /home/steeve/steeve/epfl/code/spikebias/tmp/001250/sub-002-fitted/sub-002-fitted_ses-ks25_10m.nwb -> sub-002-fitted/sub-002-fitted_ses-ks25-10m.nwb
DRY: /home/steeve/steeve/epfl/code/spikebias/tmp/001250/sub-002-fitted/sub-002-fitted_ses-hs.nwb -> sub-002-fitted/sub-002-fitted_ses-hs.nwb
DRY: /home/steeve/steeve/epfl/code/spikebias/tmp/001250/sub-002-fitted/sub-002-fitted_ses-ks3_10m.nwb -> sub-002-fitted/sub-002-fitted_ses-ks3-10m.nwb
DRY: /home/steeve/steeve/epfl/code/spikebias/tmp/001250/sub-002-fitted/sub-002-fitted_ses-ks_10m.nwb -> sub-002-fitted/sub-002-fitted_ses-ks-10m.nwb
DRY: /home/steeve/steeve/epfl/code/spikebias/tmp/001250/sub-002-fitted/sub-002-fitted_ses-ks.nwb -> sub-002-fitted/sub-002-fitted_ses-ks.nwb
DRY: /home/steeve/steeve/epfl/code/spikebias/tmp/001250/sub-002-fitted/sub-002-fitted_ses-ks4.nwb -> sub-002-fitte

2025-06-15 19:07:43,322 [    INFO] Loading metadata from 12 files
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done   3 out of  12 | elapsed:    0.6s remaining:    1.8s
[Parallel(n_jobs=-1)]: Done   5 out of  12 | elapsed:    0.6s remaining:    0.9s
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:    0.6s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done   9 out of  12 | elapsed:    0.7s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    0.8s finished
2025-06-15 19:07:44,120 [    INFO] Symlink support autodetected; setting files_mode='symlink'
2025-06-15 19:07:44,125 [    INFO] Organized 12 paths. Visit /home/steeve/steeve/epfl/code/spikebias/001250/
2025-06-15 19:07:44,125 [    INFO] Logs saved in /home/steeve/.local/state/dandi-cli/log/2025.06.15-17.07.42Z-1728586.log
2025-06-15 19:07:46,080 [    INFO] Found 25 files to consider


PATH                                           SIZE     ERRORS   PROGRESS STATUS                MESSAGE                  
dandiset.yaml                                  5.6 kB                     skipped               should be edited online  
sub-001-fitted/sub-001-fitted_ses-hs-10m.nwb                              ERROR                 File not found           
sub-001-fitted/sub-001-fitted_ses-hs.nwb                                  ERROR                 File not found           
sub-001-fitted/sub-001-fitted_ses-ks-10m.nwb                              ERROR                 File not found           
sub-001-fitted/sub-001-fitted_ses-ks.nwb                                  ERROR                 File not found           
sub-001-fitted/sub-001-fitted_ses-ks2-10m.nwb                             ERROR                 File not found           
sub-001-fitted/sub-001-fitted_ses-ks2.nwb                                 ERROR                 File not found           
sub-001-fitted/sub-001-f

2025-06-15 19:07:56,230 [    INFO] Logs saved in /home/steeve/.local/state/dandi-cli/log/2025.06.15-17.07.44Z-1728779.log
Error: File not found
